---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
data = df.copy()

In [3]:
#format 04/20/2009
s1 = data.str.extractall(r'(?P<month>\d?\d)[-/](?P<day>\d?\d)[-/](?P<year>\d{4})')
s1.reset_index(inplace=True)
data = data.drop(s1['level_0'])

In [4]:
s1['date']= pd.to_datetime(s1[['year','month','day']])

In [5]:
s1

,level_0,match,month,day,year,date
0,14,0,5,24,1990,1990-05-24
1,15,0,1,25,2011,2011-01-25
2,17,0,10,13,1976,1976-10-13
3,24,0,07,25,1984,1984-07-25
4,30,0,03,31,1985,1985-03-31
5,34,0,5,12,2012,2012-05-12
6,38,0,7,27,1986,1986-07-27
7,43,0,4,13,2002,2002-04-13
8,45,0,2,15,1998,1998-02-15
9,55,0,10,11,1987,1987-10-11


In [6]:
df[72]

'Lithium 0.25 (7/11/77).  LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherwise neg.  TSH 3.28.  BUN/Cr: 16/0.83.  Lipids unremarkable.  B12 363, Folate >20.  CBC: 4.9/36/308 Pertinent Medical Review of Systems Constitutional:\n'

In [7]:
#format 04/20/09
s2 = data.str.extractall(r'(?P<month>\d?\d)[-/](?P<day>\d?\d)[-/](?P<year>\d{2})[^\d]')
s2.reset_index(inplace=True)
data = data.drop(s2['level_0'])

In [8]:
s2['year']=s2['year'].apply(lambda x: '19'+x)

In [11]:
s2['date']= pd.to_datetime(s2[['year','month','day']])

In [12]:
s2

,level_0,match,month,day,year,date
0,0,0,03,25,1993,1993-03-25
1,1,0,6,18,1985,1985-06-18
2,2,0,7,8,1971,1971-07-08
3,3,0,9,27,1975,1975-09-27
4,4,0,2,6,1996,1996-02-06
5,5,0,7,06,1979,1979-07-06
6,6,0,5,18,1978,1978-05-18
7,7,0,10,24,1989,1989-10-24
8,8,0,3,7,1986,1986-03-07
9,9,0,4,10,1971,1971-04-10


In [13]:
#Mar-20-2009
s3 = data.str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-\.]?\s?(?P<day>\d{1,2})[-,]?\s?(?P<year>\d{4})')
s3.reset_index(inplace=True)
data = data.drop(s3['level_0'])

In [14]:
data

125    s The patient is a 44 year old married Caucasi...
126    .10 Sep 2004 - Intake at EEC for IOP but did n...
127    see above and APS eval of 26 May 1982 Social H...
128    Tbooked for intake appointment at Sierra Vista...
129                    06 May 1972 SOS-10 Total Score:\n
130      25 Oct 1987 Total time of visit (in minutes):\n
131                    14 Oct 1996 SOS-10 Total Score:\n
132    30 Nov 2007 CPT Code: 90801 - Psychiatric Diag...
133    h missed intake office visit on 28 June 1994 a...
134                    14 Jan 1981 SOS-10 Total Score:\n
135      .On 10 Oct 1985 patient began FMLA from work.\n
136    11 February 1985 CPT Code: 90801 - Psychiatric...
137    7intake for follow up treatment at Anson Gener...
138    s The patient is a 49 year old married Canadia...
139    ssee 21 Oct 2012 Schroder Hospital discharge s...
140                   14 Feb 1995 Primary Care Doctor:\n
141                    30 May 2016 SOS-10 Total Score:\n
142    22 Jan 1996 @ 11 AMCommu

In [16]:
s3

,level_0,match,month,day,year
0,194,0,Apr,11,1990
1,195,0,May,30,2001
2,196,0,Feb,18,1994
3,197,0,Feb,18,1981
4,198,0,Oct,11,2013
5,199,0,Jan,24,1986
6,200,0,Jul,26,1978
7,201,0,Dec,23,1999
8,202,0,May,15,1989
9,203,0,Sep,06,1995


In [17]:
month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                   'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                   'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                   'July': 7, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
s3.replace({"month": month_dict}, inplace=True)

In [19]:
s3['date']= pd.to_datetime(s3[['year','month','day']])

In [20]:
s3

,level_0,match,month,day,year,date
0,194,0,4,11,1990,1990-04-11
1,195,0,5,30,2001,2001-05-30
2,196,0,2,18,1994,1994-02-18
3,197,0,2,18,1981,1981-02-18
4,198,0,10,11,2013,2013-10-11
5,199,0,1,24,1986,1986-01-24
6,200,0,7,26,1978,1978-07-26
7,201,0,12,23,1999,1999-12-23
8,202,0,5,15,1989,1989-05-15
9,203,0,9,06,1995,1995-09-06


In [21]:
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
s4 = data.str.extractall(r'(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\.,]? (?P<year>\d{4})')
s4.reset_index(inplace=True)
data = data.drop(s4['level_0'])

In [23]:
month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                   'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                   'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                   'July': 7, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
s4.replace({"month": month_dict}, inplace=True)

In [24]:
s4['date']= pd.to_datetime(s4[['year','month','day']])

In [25]:
s4

,level_0,match,day,month,year,date
0,125,0,24,1,2001,2001-01-24
1,126,0,10,9,2004,2004-09-10
2,127,0,26,5,1982,1982-05-26
3,128,0,28,6,2002,2002-06-28
4,129,0,06,5,1972,1972-05-06
5,130,0,25,10,1987,1987-10-25
6,131,0,14,10,1996,1996-10-14
7,132,0,30,11,2007,2007-11-30
8,133,0,28,6,1994,1994-06-28
9,134,0,14,1,1981,1981-01-14


In [26]:
#Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
sa = data.str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?P<day>\d{1,2})[a-z]*, (?P<year>\d{4})')
sa.reset_index(inplace=True)

In [28]:
sa

,index,month,day,year


In [29]:
#Feb 2009; Sep 2009; Oct 2010
s5 = data.str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,? (?P<year>\d{4})')
s5.reset_index(inplace=True)
data = data.drop(s5['level_0'])

In [35]:
s5['day'] = 1

In [32]:
month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                   'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                   'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                   'July': 7, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
s5.replace({"month": month_dict}, inplace=True)

In [37]:
s5['date']= pd.to_datetime(s5[['year','month','day']])

In [38]:
s5

,level_0,match,month,year,day,date
0,228,0,9,1985,1,1985-09-01
1,229,0,6,2011,1,2011-06-01
2,230,0,5,1986,1,1986-05-01
3,231,0,5,2016,1,2016-05-01
4,232,0,7,1977,1,1977-07-01
5,233,0,7,1990,1,1990-07-01
6,234,0,7,2003,1,2003-07-01
7,235,0,10,2015,1,2015-10-01
8,236,0,5,1995,1,1995-05-01
9,237,0,2,1976,1,1976-02-01


In [39]:
#6/2008; 12/2009
s6 = data.str.extractall(r'(?P<month>\d{1,2})/(?P<year>\d{4})')
s6.reset_index(inplace=True)
data = data.drop(s6['level_0'])

In [41]:
s6['day'] = 1

In [42]:
s6['date']= pd.to_datetime(s6[['year','month','day']])

In [43]:
s6

,level_0,match,month,year,day,date
0,343,0,6,1998,1,1998-06-01
1,344,0,6,2005,1,2005-06-01
2,345,0,10,1973,1,1973-10-01
3,346,0,9,2005,1,2005-09-01
4,347,0,03,1980,1,1980-03-01
5,348,0,12,2005,1,2005-12-01
6,349,0,5,1987,1,1987-05-01
7,350,0,5,2004,1,2004-05-01
8,351,0,8,1974,1,1974-08-01
9,352,0,3,1986,1,1986-03-01


In [44]:
#2009; 2010
s7 = data.str.extractall(r'(?P<year>\d{4})')
s7.reset_index(inplace=True)
data = data.drop(s7['level_0'])

In [45]:
s7

,level_0,match,year
0,455,0,1984
1,456,0,2000
2,457,0,2001
3,458,0,1982
4,459,0,1998
5,460,0,2012
6,461,0,1991
7,462,0,1988
8,463,0,2014
9,464,0,2016


In [46]:
s7['month'] = 1
s7['day'] = 1

In [47]:
s7['date']= pd.to_datetime(s7[['year','month','day']])

In [48]:
s7

,level_0,match,year,month,day,date
0,455,0,1984,1,1,1984-01-01
1,456,0,2000,1,1,2000-01-01
2,457,0,2001,1,1,2001-01-01
3,458,0,1982,1,1,1982-01-01
4,459,0,1998,1,1,1998-01-01
5,460,0,2012,1,1,2012-01-01
6,461,0,1991,1,1,1991-01-01
7,462,0,1988,1,1,1988-01-01
8,463,0,2014,1,1,2014-01-01
9,464,0,2016,1,1,2016-01-01


In [49]:
data

Series([], dtype: object)

In [50]:
s1.head()

,level_0,match,month,day,year,date
0,14,0,5,24,1990,1990-05-24
1,15,0,1,25,2011,2011-01-25
2,17,0,10,13,1976,1976-10-13
3,24,0,07,25,1984,1984-07-25
4,30,0,03,31,1985,1985-03-31


In [51]:
s2.head()

,level_0,match,month,day,year,date
0,0,0,03,25,1993,1993-03-25
1,1,0,6,18,1985,1985-06-18
2,2,0,7,8,1971,1971-07-08
3,3,0,9,27,1975,1975-09-27
4,4,0,2,6,1996,1996-02-06


In [52]:
s3.head()

,level_0,match,month,day,year,date
0,194,0,4,11,1990,1990-04-11
1,195,0,5,30,2001,2001-05-30
2,196,0,2,18,1994,1994-02-18
3,197,0,2,18,1981,1981-02-18
4,198,0,10,11,2013,2013-10-11


In [53]:
s4.head()

,level_0,match,day,month,year,date
0,125,0,24,1,2001,2001-01-24
1,126,0,10,9,2004,2004-09-10
2,127,0,26,5,1982,1982-05-26
3,128,0,28,6,2002,2002-06-28
4,129,0,06,5,1972,1972-05-06


In [54]:
s5.head()

,level_0,match,month,year,day,date
0,228,0,9,1985,1,1985-09-01
1,229,0,6,2011,1,2011-06-01
2,230,0,5,1986,1,1986-05-01
3,231,0,5,2016,1,2016-05-01
4,232,0,7,1977,1,1977-07-01


In [55]:
s6.head()

,level_0,match,month,year,day,date
0,343,0,6,1998,1,1998-06-01
1,344,0,6,2005,1,2005-06-01
2,345,0,10,1973,1,1973-10-01
3,346,0,9,2005,1,2005-09-01
4,347,0,03,1980,1,1980-03-01


In [56]:
s7.head()

,level_0,match,year,month,day,date
0,455,0,1984,1,1,1984-01-01
1,456,0,2000,1,1,2000-01-01
2,457,0,2001,1,1,2001-01-01
3,458,0,1982,1,1,1982-01-01
4,459,0,1998,1,1,1998-01-01


In [73]:
result = pd.concat([s1[['level_0','date']],s2[['level_0','date']],s3[['level_0','date']],s4[['level_0','date']],
                    s5[['level_0','date']],s6[['level_0','date']],s7[['level_0','date']]])

In [74]:
date = result.sort_values('level_0').reset_index()['date']

In [75]:
date

0     1993-03-25
1     1985-06-18
2     1971-07-08
3     1975-09-27
4     1996-02-06
5     1979-07-06
6     1978-05-18
7     1989-10-24
8     1986-03-07
9     1971-04-10
10    1985-05-11
11    1975-04-09
12    1998-08-01
13    1972-01-26
14    1990-05-24
15    2011-01-25
16    1982-04-12
17    1976-10-13
18    1998-04-24
19    1977-05-21
20    1998-07-21
21    1979-10-21
22    1990-03-03
23    1976-02-11
24    1984-07-25
25    1982-04-13
26    1989-09-22
27    1976-09-02
28    1971-09-12
29    1986-10-24
         ...    
470   1983-01-01
471   1999-01-01
472   2010-01-01
473   1975-01-01
474   1972-01-01
475   2015-01-01
476   1989-01-01
477   1994-01-01
478   1993-01-01
479   1996-01-01
480   2013-01-01
481   1974-01-01
482   1990-01-01
483   1995-01-01
484   2004-01-01
485   1987-01-01
486   1973-01-01
487   1992-01-01
488   1977-01-01
489   1985-01-01
490   2007-01-01
491   2009-01-01
492   1986-01-01
493   1978-01-01
494   2002-01-01
495   1979-01-01
496   2006-01-01
497   2008-01-

In [80]:
index = date.sort_values()

In [82]:
index.index

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 427, 231, 141, 186, 161, 413],
           dtype='int64', length=500)

In [84]:
final = pd.Series(index.index)

In [3]:
def date_sorter():
    
    
    return final